**The objective of this project is to practice data manipulation with Python and plotting data onto a map by using geolocation data from a CSV file, the dataset we are going to use for this project contains information about the earthquakes registered in Costa Rica from 1/31/2021 to 3/1/2021 with 40 data points describing: date, time, magnitud, deepth, location, latitude and longitude.*

In [1]:
#Imports of the modules we are going to use
import os
import pandas as pd

In [2]:
#Instead of reading the data from the csv file, let's retrieve the data from the site itself following this URL:http://www.ovsicori.una.ac.cr/sistemas/sentidos_map/indexleqs.php
table_data = pd.read_html("http://www.ovsicori.una.ac.cr/sistemas/sentidos_map/indexleqs.php")

In [3]:
#Convert the data for a dataframe
df = pd.DataFrame(data=table_data[0])
df.head(10)

,Fecha,Hora,Magnitud,Profundidad,Latitud,Longitud,Localizacion,Revisado,Autor,Mapa
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-05,03:59:57,2.4,0.0,7.9336,-82.6912,29.2 km al Sureste de Pavon de Puntarenas,No,olDbg,mapa
2,2021-05-05,03:05:39,2.3,98.0,8.5829,-83.2383,6.0 km al Sur de Golfito de Puntarenas,No,-,mapa
3,2021-05-05,02:13:05,0.9,0.0,9.7287,-84.1483,2.5 km al Suroeste de Vuelta de Jorco de San Jose,No,olDbg,mapa
4,2021-05-05,01:15:48,2.7,21.0,9.2084,-82.5190,21.7 km al Suroeste de - de Panama,No,olDbg,mapa
5,2021-05-05,00:54:37,3.1,15.0,9.1242,-85.5378,68.5 km al Suroeste de Cobano de Puntarenas,No,olDbg,mapa
6,2021-05-05,00:50:53,2.0,18.0,9.5667,-84.7694,15.2 km al Suroeste de Jaco de Puntarenas,No,olDbg,mapa
7,2021-05-05,00:05:44,2.9,21.0,10.7969,-85.7022,11.3 km al Noroeste de Nacascolo de Guanacaste,No,olDbg,mapa
8,2021-05-04,21:41:26,2.2,17.0,8.7956,-83.7550,14.0 km al Noroeste de Sierpe de Puntarenas,No,olDbg,mapa
9,2021-05-04,18:22:52,2.4,50.0,10.1448,-85.4857,1.5 km al Sureste de Nicoya de Guanacaste,Si,olMlMwp,mapa


In [5]:
df.dropna()

,Fecha,Hora,Magnitud,Profundidad,Latitud,Longitud,Localizacion,Revisado,Autor,Mapa
1,2021-05-05,03:59:57,2.4,0.0,7.9336,-82.6912,29.2 km al Sureste de Pavon de Puntarenas,No,olDbg,mapa
2,2021-05-05,03:05:39,2.3,98.0,8.5829,-83.2383,6.0 km al Sur de Golfito de Puntarenas,No,-,mapa
3,2021-05-05,02:13:05,0.9,0.0,9.7287,-84.1483,2.5 km al Suroeste de Vuelta de Jorco de San Jose,No,olDbg,mapa
4,2021-05-05,01:15:48,2.7,21.0,9.2084,-82.5190,21.7 km al Suroeste de - de Panama,No,olDbg,mapa
5,2021-05-05,00:54:37,3.1,15.0,9.1242,-85.5378,68.5 km al Suroeste de Cobano de Puntarenas,No,olDbg,mapa
6,2021-05-05,00:50:53,2.0,18.0,9.5667,-84.7694,15.2 km al Suroeste de Jaco de Puntarenas,No,olDbg,mapa
7,2021-05-05,00:05:44,2.9,21.0,10.7969,-85.7022,11.3 km al Noroeste de Nacascolo de Guanacaste,No,olDbg,mapa
8,2021-05-04,21:41:26,2.2,17.0,8.7956,-83.7550,14.0 km al Noroeste de Sierpe de Puntarenas,No,olDbg,mapa
9,2021-05-04,18:22:52,2.4,50.0,10.1448,-85.4857,1.5 km al Sureste de Nicoya de Guanacaste,Si,olMlMwp,mapa
10,2021-05-04,16:12:15,4.2,15.0,13.8949,-85.7602,45.8 km al Este de - de Nicaragua,No,or,mapa


In [6]:
#We dropped the column "Localizacion", "Revisado","Autor","Mapa", since we are going to use the geopy library to find out the location based on the coordinates
df = df.drop(columns=["Localizacion", "Revisado","Autor","Mapa"])

In [7]:
#Confirm the column "Localizacion" was dropped
df.head(10)

,Fecha,Hora,Magnitud,Profundidad,Latitud,Longitud
1,2021-05-05,03:59:57,2.4,0.0,7.9336,-82.6912
2,2021-05-05,03:05:39,2.3,98.0,8.5829,-83.2383
3,2021-05-05,02:13:05,0.9,0.0,9.7287,-84.1483
4,2021-05-05,01:15:48,2.7,21.0,9.2084,-82.5190
5,2021-05-05,00:54:37,3.1,15.0,9.1242,-85.5378
6,2021-05-05,00:50:53,2.0,18.0,9.5667,-84.7694
7,2021-05-05,00:05:44,2.9,21.0,10.7969,-85.7022
8,2021-05-04,21:41:26,2.2,17.0,8.7956,-83.7550
9,2021-05-04,18:22:52,2.4,50.0,10.1448,-85.4857
10,2021-05-04,16:12:15,4.2,15.0,13.8949,-85.7602


In [23]:
#let's rename the columns from Spanish to English
df = df.rename(columns={"Fecha":"Date","Hora":"Local Time","Magnitud":"Magnitude","Profundidad":"Deepth(KM)", "Latitud":"Latitude", "Longitud":"Longitude"})

In [24]:
#Confirmed our columns were renamed correctly
df.head(10)

,Date,Local Time,Magnitude,Deepth(KM),Latitude,Longitude,Coordinates,Location
1,2021-05-05,03:59:57,2.4,0.0,7.9336,-82.6912,"7.9336, -82.6912","(Chiriquí, Panamá, (8.139963649999999, -82.259..."
2,2021-05-05,03:05:39,2.3,98.0,8.5829,-83.2383,"8.5829, -83.2383","(Provincia Puntarenas, Costa Rica, (9.13740299..."
3,2021-05-05,02:13:05,0.9,0.0,9.7287,-84.1483,"9.7287, -84.1483","(Cangrejal, Cantón Acosta, Provincia San José,..."
4,2021-05-05,01:15:48,2.7,21.0,9.2084,-82.5190,"9.2084, -82.519","(Distrito Changuinola, Bocas del Toro, Panamá,..."
5,2021-05-05,00:54:37,3.1,15.0,9.1242,-85.5378,"9.1242, -85.5378",None
6,2021-05-05,00:50:53,2.0,18.0,9.5667,-84.7694,"9.5667, -84.7694",None
7,2021-05-05,00:05:44,2.9,21.0,10.7969,-85.7022,"10.7969, -85.7022","(Provincia Guanacaste, Costa Rica, (10.4313298..."
8,2021-05-04,21:41:26,2.2,17.0,8.7956,-83.7550,"8.7956, -83.755","(Provincia Puntarenas, Costa Rica, (9.13740299..."
9,2021-05-04,18:22:52,2.4,50.0,10.1448,-85.4857,"10.1448, -85.4857","(Nicoya, Cantón Nicoya, Provincia Guanacaste, ..."
10,2021-05-04,16:12:15,4.2,15.0,13.8949,-85.7602,"13.8949, -85.7602","(Wiwilí de Jinotega (Municipio), Jinotega, 667..."


In [10]:
#Since the geopy reverse() function only accepts an unique string containing the latitude and longitude for the coordinates, then; let's create a column called Coordinates and let's concatenate the values for latitude and longitude and convert them to strings.
df["Coordinates"] = df["Latitude"].astype(str) + ", " + df["Longitude"].astype(str)

In [11]:
#Now we can see the column Coordinates holding both latitude and longitude but a strings
df.head()

,Date,Hora,Magnitude,Deepth(KM),Latitude,Longitude,Coordinates
1,2021-05-05,03:59:57,2.4,0.0,7.9336,-82.6912,"7.9336, -82.6912"
2,2021-05-05,03:05:39,2.3,98.0,8.5829,-83.2383,"8.5829, -83.2383"
3,2021-05-05,02:13:05,0.9,0.0,9.7287,-84.1483,"9.7287, -84.1483"
4,2021-05-05,01:15:48,2.7,21.0,9.2084,-82.5190,"9.2084, -82.519"
5,2021-05-05,00:54:37,3.1,15.0,9.1242,-85.5378,"9.1242, -85.5378"


Let's use the Geopy library to figure out from the coordinates the location where the earthquake was registered
https://pypi.org/project/geopy/


In [16]:
#Importing the geopy library
from geopy.geocoders import Nominatim

In [13]:
#This empty list will hold each location found based on the coordinates
location = []

In [17]:
geolocator = Nominatim(user_agent="Earthquakes in CR")

In [15]:
#We loop through the rows in the dataset, and call the reverse() function to find the location based on the coordinates
for index, row in df.iterrows():
    location.append(str(geolocator.reverse(row["Coordinates"])))
    print(str(index) + " " + location[index])

0 None
1 None
2 Chiriquí, Panamá
3 Chiriquí, Panamá
4 Carillo, Provincia Guanacaste, 50302, Costa Rica
5 Vía 224, Dulce Nombre, Cantón Cartago, Provincia Cartago, 30109, Costa Rica
6 Finca Guayacán, Rodolfo Aguilar Delgado, Distrito Barú, Chiriquí, Panamá
7 Avenida 42, Precario Barrio Nuevo, Curridabat, Cantón Curridabat, Provincia San José, 11801, Costa Rica
8 PALI, Avenida 88, Barrio Fallas, Desamparados, Cantón Desamparados, Provincia San José, 10301, Costa Rica
9 San José, Rivas, Cantón Pérez Zeledón, Provincia San José, 11904, Costa Rica
10 Flor del Roble de Coto Brus, Gutiérrez Braun, Cantón Coto Brus, Provincia Puntarenas, 60806, Costa Rica
11 Barú, Cantón Pérez Zeledón, Provincia San José, 11909, Costa Rica
12 Provincia Puntarenas, Costa Rica
13 Calle 47, Lomas de San Francisco, San Francisco de Dos Ríos, San José, Cantón San José, Provincia San José, 10106, Costa Rica
14 La Rosticería, Vía 210, Barrio La Colina, San Antonio, Cantón Desamparados, Provincia San José, 11804, Cost

In [19]:
df["Location"] = df["Coordinates"].apply(geolocator.reverse)

In [20]:
df.head(10)

,Date,Hora,Magnitude,Deepth(KM),Latitude,Longitude,Coordinates,Location
1,2021-05-05,03:59:57,2.4,0.0,7.9336,-82.6912,"7.9336, -82.6912","(Chiriquí, Panamá, (8.139963649999999, -82.259..."
2,2021-05-05,03:05:39,2.3,98.0,8.5829,-83.2383,"8.5829, -83.2383","(Provincia Puntarenas, Costa Rica, (9.13740299..."
3,2021-05-05,02:13:05,0.9,0.0,9.7287,-84.1483,"9.7287, -84.1483","(Cangrejal, Cantón Acosta, Provincia San José,..."
4,2021-05-05,01:15:48,2.7,21.0,9.2084,-82.5190,"9.2084, -82.519","(Distrito Changuinola, Bocas del Toro, Panamá,..."
5,2021-05-05,00:54:37,3.1,15.0,9.1242,-85.5378,"9.1242, -85.5378",None
6,2021-05-05,00:50:53,2.0,18.0,9.5667,-84.7694,"9.5667, -84.7694",None
7,2021-05-05,00:05:44,2.9,21.0,10.7969,-85.7022,"10.7969, -85.7022","(Provincia Guanacaste, Costa Rica, (10.4313298..."
8,2021-05-04,21:41:26,2.2,17.0,8.7956,-83.7550,"8.7956, -83.755","(Provincia Puntarenas, Costa Rica, (9.13740299..."
9,2021-05-04,18:22:52,2.4,50.0,10.1448,-85.4857,"10.1448, -85.4857","(Nicoya, Cantón Nicoya, Provincia Guanacaste, ..."
10,2021-05-04,16:12:15,4.2,15.0,13.8949,-85.7602,"13.8949, -85.7602","(Wiwilí de Jinotega (Municipio), Jinotega, 667..."


Now, that we have our data set ready, let's create some visualization using plotly:https://plotly.com/python/mapbox-layers/ 

In [21]:
import plotly.express as px

In [25]:
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_name="Date", hover_data=["Local Time", "Magnitude", "Deepth(KM)", "Coordinates"], color_discrete_sequence=["blue"])
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [20]:
fig.write_html("C:\\Users\jocerdas\\OneDrive - Microsoft\Documents\\Python\\Python Projects\\earthquakers\\cr_earthquakes\\cr_earthquakes.html")